In [8]:
# 01_move_sheet1_to_outputs.py
from openpyxl import load_workbook

# ===== 설정 =====
FILE_PATH      = "1003.xlsx"
SRC_SHEET      = "Sheet1"
DST_BASE       = "output"
SRC_START_COL  = 2           # Sheet1에서 B열부터 복사
DST_START_ROW  = 857_842     # output에서 시작 행
DST_START_COL  = 2           # output에서 B열부터
EXCEL_MAX_ROWS = 1_048_576
EXCEL_MAX_COLS = 16_384
# =================

def main():
    wb = load_workbook(FILE_PATH)
    if SRC_SHEET not in wb.sheetnames:
        raise ValueError(f"시트 없음: {SRC_SHEET}")
    if DST_BASE not in wb.sheetnames:
        raise ValueError(f"시트 없음: {DST_BASE}")

    ws_src = wb[SRC_SHEET]
    max_row = ws_src.max_row
    max_col = ws_src.max_column

    # 열 한도 체크
    needed_cols = DST_START_COL + (max_col - SRC_START_COL)
    if needed_cols > EXCEL_MAX_COLS:
        raise ValueError(f"붙여넣기 후 열({needed_cols})이 엑셀 최대({EXCEL_MAX_COLS})를 초과합니다.")

    # 복사할 총 행 수 (헤더 포함 전체)
    remain = max_row
    src_row_ptr = 1

    # 1) 첫 대상 시트: output 의 지정 시작행부터 가능한 만큼
    plans = []  # [(sheet_name, dst_row_start, src_row_start, rows)]
    room_first = EXCEL_MAX_ROWS - DST_START_ROW + 1
    use_first = max(0, min(remain, room_first))
    if use_first > 0:
        plans.append((DST_BASE, DST_START_ROW, src_row_ptr, use_first))
        remain      -= use_first
        src_row_ptr += use_first

    # 2) 넘치는 건 output_2, output_3 ... 로 1행부터 이어붙이기
    idx = 2
    while remain > 0:
        name = f"{DST_BASE}_{idx}"
        if name not in wb.sheetnames:
            wb.create_sheet(name)
        use = min(remain, EXCEL_MAX_ROWS)
        plans.append((name, 1, src_row_ptr, use))
        remain      -= use
        src_row_ptr += use
        idx         += 1

    # 실행
    for sheet_name, dst_row0, src_row0, rows in plans:
        ws_dst = wb[sheet_name]
        for dr in range(rows):  # 0..rows-1
            r_src = src_row0 + dr
            r_dst = dst_row0 + dr
            for c in range(SRC_START_COL, max_col + 1):
                ws_dst.cell(
                    row=r_dst,
                    column=DST_START_COL + (c - SRC_START_COL),
                    value=ws_src.cell(row=r_src, column=c).value
                )

    wb.save(FILE_PATH)  # 같은 파일에 반영 (원하면 다른 이름으로 저장)
    wb.close()

    print("복사 계획:")
    for s, drow, srow, rows in plans:
        print(f"  - {s}: dst_row={drow}, src_row={srow}, rows={rows}")
    print(f"완료: {FILE_PATH}")

if __name__ == "__main__":
    main()


복사 계획:
  - output: dst_row=857842, src_row=1, rows=190735
  - output_2: dst_row=1, src_row=190736, rows=96739
완료: 1003.xlsx


### 30일 룰

In [14]:
# 02_dedup_30days_from_outputs.py
import re
import pandas as pd

FILE_PATH   = "1003.xlsx"             # 원본 파일에서 output* 읽음
SHEET_PATT  = r"^output(_\d+)?$"
OUT_PATH    = "dedup_30d.xlsx"
BASE_SHEET  = "dedup_30d"

KW_COL = "뉴스 키워드 후보"
CO_COL = "회사명"
DT_COL = "뉴스 보도날짜(YYYYMMDD)"
URL_COL = "URL"   # 있으면

WINDOW_DAYS = 30
EXCEL_MAX_ROWS = 1_048_576

def write_split(df: pd.DataFrame, path: str, base: str):
    with pd.ExcelWriter(path, engine="openpyxl") as w:
        if len(df) == 0:
            df.head(0).to_excel(w, index=False, sheet_name=base)
            return
        start, part = 0, 1
        while start < len(df):
            end  = min(start + EXCEL_MAX_ROWS - 1, len(df))
            name = base if part == 1 else f"{base}_{part}"
            df.iloc[start:end].to_excel(w, index=False, sheet_name=name)
            start = end
            part += 1

def main():
    xl = pd.ExcelFile(FILE_PATH)
    sheets = [s for s in xl.sheet_names if re.match(SHEET_PATT, s)]
    if not sheets:
        raise ValueError("output / output_2 ... 시트를 찾지 못했습니다.")

    frames = [pd.read_excel(FILE_PATH, sheet_name=s, dtype=str) for s in sheets]
    df = pd.concat(frames, ignore_index=True)
    df = df.rename(columns=lambda x: str(x).strip())

    # 필수 컬럼 체크
    for col in [KW_COL, CO_COL, DT_COL]:
        if col not in df.columns:
            raise ValueError(f"필수 열 누락: {col} (D=뉴스 키워드 후보, K=회사, G=YYYYMMDD)")

    work = df.copy()
    work["__kw"] = work[KW_COL].fillna("").str.strip().str.lower()
    work["__co"] = work[CO_COL].fillna("").str.strip().str.lower()
    work["__date"] = pd.to_datetime(work[DT_COL].astype(str).str.strip(),
                                    format="%Y%m%d", errors="coerce")

    # 완전 중복(키·회사·날짜·URL) 제거
    subset = ["__kw", "__co", "__date"]
    if URL_COL in work.columns:
        subset.append(URL_COL)
    work = work.drop_duplicates(subset=subset, keep="first")

    work = work.sort_values(["__kw", "__co", "__date"], kind="stable")

    def keep_rule(g: pd.DataFrame) -> pd.DataFrame:
        keep_idx, last = [], pd.Timestamp.min
        for idx, row in g.iterrows():
            d = row["__date"]
            if pd.isna(d):  # 날짜 없으면 유지(필요하면 제외로 변경)
                keep_idx.append(idx); continue
            if d >= last + pd.Timedelta(days=WINDOW_DAYS):
                keep_idx.append(idx); last = d
        return g.loc[keep_idx]

    out = work.groupby(["__kw", "__co"], group_keys=False).apply(keep_rule)
    out = out.drop(columns=["__kw", "__co", "__date"]).reset_index(drop=True)

    write_split(out, OUT_PATH, BASE_SHEET)
    print(f"30일 규칙: {len(work)} → {len(out)}행 저장 → {OUT_PATH}")

if __name__ == "__main__":
    main()


C:\Users\user\AppData\Local\Temp\ipykernel_26744\1357784805.py:70: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  out = work.groupby(["__kw", "__co"], group_keys=False).apply(keep_rule)


30일 규칙: 496763 → 192944행 저장 → dedup_30d.xlsx


#### (데코 평산 국제개발 도움 우방 우영 부흥 세신 자강) 제거 및 [Who is ?]로 시작하는 기사 없애기

In [18]:
# 04_append_cleaned_sheets_into_same_file.py
import re
import pandas as pd
from openpyxl import load_workbook

IN_PATH     = "dedup_30d.xlsx"            # 기존 파일에 "시트 추가"로 저장
SHEET_PATT  = r"^dedup_30d(_\d+)?$"

# 컬럼명
KW_COL  = "뉴스 키워드 후보"
CO_COL  = "회사명"
DT_COL  = "뉴스 보도날짜(YYYYMMDD)"
URL_COL = "URL"  # 없어도 됨

# 제외(블랙리스트) 회사명 (정확히 일치)
BLACKLIST = {"데코", "평산", "국제개발", "도움", "우방", "우영", "부흥", "세신", "자강", "대국"}

EXCEL_MAX_ROWS = 1_048_576

def unique_sheet_name(existing: set, base: str) -> str:
    """기존 시트 목록과 충돌하지 않게 고유한 시트명 생성."""
    if base not in existing:
        existing.add(base)
        return base
    i = 2
    while True:
        cand = f"{base}_{i}"
        if cand not in existing:
            existing.add(cand)
            return cand
        i += 1

def write_split_append(wb, df: pd.DataFrame, base: str):
    """
    같은 파일(워크북)에 시트를 '추가'로 기록.
    행이 많으면 base, base_2, base_3 ... 로 분할 저장.
    """
    existing = set(wb.sheetnames)
    with pd.ExcelWriter(IN_PATH, engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
        writer.book = wb
        writer.sheets = {ws.title: ws for ws in wb.worksheets}

        n = len(df)
        if n == 0:
            name = unique_sheet_name(existing, base)
            df.head(0).to_excel(writer, index=False, sheet_name=name)
            return

        start = 0
        part  = 1
        while start < n:
            end  = min(start + EXCEL_MAX_ROWS - 1, n)
            name = unique_sheet_name(existing, base if part == 1 else f"{base}_{part}")
            df.iloc[start:end].to_excel(writer, index=False, sheet_name=name)
            start = end
            part += 1

def get_h_colname(df: pd.DataFrame) -> str:
    """H열(8번째, 0-based 7)의 실제 컬럼명 반환."""
    if df.shape[1] < 8:
        raise ValueError(f"H열을 찾을 수 없습니다. (열 개수 {df.shape[1]} < 8)")
    return df.columns[7]

def main():
    # 1) 원본 시트들 읽기
    xl = pd.ExcelFile(IN_PATH)
    sheets = [s for s in xl.sheet_names if re.match(SHEET_PATT, s)]
    if not sheets:
        raise ValueError("dedup_30d / dedup_30d_2 ... 시트를 찾지 못했습니다.")
    frames = [pd.read_excel(IN_PATH, sheet_name=s, dtype=str) for s in sheets]
    df = pd.concat(frames, ignore_index=True)

    # 2) 컬럼명 트림 + 필수 컬럼 검사
    df = df.rename(columns=lambda x: str(x).strip())
    for col in [KW_COL, CO_COL, DT_COL]:
        if col not in df.columns:
            raise ValueError(f"필수 열 누락: {col}")

    # 3) 블랙리스트 분리
    co_norm = df[CO_COL].fillna("").str.strip()
    is_removed = co_norm.isin(BLACKLIST)
    removed_by_company = df.loc[is_removed].copy()
    kept = df.loc[~is_removed].copy()

    # 4) kept에서 H열 "[Who is ?]" 시작 분리
    h_col = get_h_colname(kept)
    whois_mask = kept[h_col].fillna("").str.strip().str.startswith("[Who is ?]")
    kept_clean = kept.loc[~whois_mask].copy()
    kept_whois_removed = kept.loc[whois_mask].copy()

    # 5) 같은 파일에 "시트 추가"로 저장
    wb = load_workbook(IN_PATH)  # 기존 워크북 로드
    write_split_append(wb, kept,                 "dedup_30d_kept")
    write_split_append(wb, removed_by_company,   "dedup_30d_removed")
    write_split_append(wb, kept_clean,           "dedup_30d_kept_clean")
    write_split_append(wb, kept_whois_removed,   "dedup_30d_kept_whois_removed")
    wb.save(IN_PATH)
    wb.close()

    print(f"총 {len(df)}행 → 블랙리스트 제외 {len(removed_by_company)}행, 남김 {len(kept)}행")
    print(f"kept 중 H열 '[Who is ?]' 시작 제외 {len(kept_whois_removed)}행, 최종 {len(kept_clean)}행")
    print(f"같은 파일에 시트 추가 완료 → {IN_PATH}")

if __name__ == "__main__":
    main()


AttributeError: property 'book' of 'OpenpyxlWriter' object has no setter

### 월 하나씩만

In [ ]:
# 03_pick_monthly_one_from_outputs.py  (월(MM) 기준 (키워드,회사) 월 1건)
import re
import pandas as pd

IN_PATH     = "1003.xlsx"                 # 입력 엑셀
SHEET_PATT  = r"^output(_\d+)?$"          # output, output_2, output_3 ... 전부 읽기
OUT_PATH    = "monthly_one.xlsx"          # 결과 파일
BASE_SHEET  = "monthly_one"               # 결과 시트(여러 개로 분할 저장될 수 있음)
EXCEL_MAX_ROWS = 1_048_576

KW_COL = "뉴스 키워드 후보"
CO_COL = "회사명"
DT_COL = "뉴스 보도날짜(YYYYMMDD)"
URL_COL = "URL"   # 있으면

# 폴백: 엑셀 열 위치 (0-based index)  D=3, G=6, K=10, J=9
FALLBACK_POS = {"KW": 3, "DT": 6, "CO": 10, "URL": 9}

def resolve_col(df: pd.DataFrame, preferred: str, fallback_idx: int) -> str:
    """컬럼명이 preferred면 그대로, 없으면 열 위치로 폴백하여 실제 df 컬럼명을 반환."""
    cols = list(df.columns)
    if preferred in cols:
        return preferred
    if 0 <= fallback_idx < len(cols):
        return cols[fallback_idx]
    raise KeyError(f"컬럼 해석 실패: '{preferred}'도 없고, 인덱스 {fallback_idx}도 범위를 벗어남. cols={cols}")

def write_split(df: pd.DataFrame, path: str, base: str):
    with pd.ExcelWriter(path, engine="openpyxl") as w:
        if len(df) == 0:
            df.head(0).to_excel(w, index=False, sheet_name=base); return
        start, part = 0, 1
        while start < len(df):
            end  = min(start + EXCEL_MAX_ROWS - 1, len(df))
            name = base if part == 1 else f"{base}_{part}"
            df.iloc[start:end].to_excel(w, index=False, sheet_name=name)
            start = end; part += 1

def main():
    # 1) output, output_2 ... 시트 모아 읽기
    xl = pd.ExcelFile(IN_PATH)
    sheets = [s for s in xl.sheet_names if re.match(SHEET_PATT, s)]
    if not sheets:
        raise ValueError("output / output_2 ... 시트를 찾지 못했습니다.")

    frames = [pd.read_excel(IN_PATH, sheet_name=s, dtype=str) for s in sheets]
    df = pd.concat(frames, ignore_index=True)

    # 2) 컬럼명 해석 (한글 헤더 우선, 없으면 D/G/K/J 위치로 폴백)
    KW_COL  = resolve_col(df, PREF_KW,  FALLBACK_POS["KW"])
    CO_COL  = resolve_col(df, PREF_CO,  FALLBACK_POS["CO"])
    DT_COL  = resolve_col(df, PREF_DT,  FALLBACK_POS["DT"])
    URL_COL = None
    try:
        URL_COL = resolve_col(df, PREF_URL, FALLBACK_POS["URL"])
    except KeyError:
        pass  # URL 없으면 None

    # 3) 날짜 파싱
    work = df.copy()
    work["__date"] = pd.to_datetime(
        work[DT_COL].astype(str).str.strip(),
        format="%Y%m%d",
        errors="coerce"
    )

    # 4) 완전 중복 제거: (키워드, 회사, 날짜[, URL]) 동일 시 1건만
    subset = [KW_COL, CO_COL, "__date"]
    if URL_COL and (URL_COL in work.columns):
        subset.append(URL_COL)
    work = work.drop_duplicates(subset=subset, keep="first")

    # 5) 월(MM) 기준으로 (키워드, 회사)당 월 1건 선택 (그 달 '가장 이른' 기사)
    work["__ym"] = work["__date"].dt.to_period("M")
    picked_idx = []
    for (_, _, ym), g in work.dropna(subset=["__date", "__ym"]).groupby([KW_COL, CO_COL, "__ym"]):
        idx = g["__date"].idxmin()   # 최신을 원하면 idxmax()로 변경
        picked_idx.append(idx)

    out = (work.loc[picked_idx]
           .sort_values([KW_COL, CO_COL, "__date"])
           .drop(columns=["__date", "__ym"])
           .reset_index(drop=True))

    # 6) 저장 (행 많으면 monthly_one_2, _3 ... 로 분할)
    write_split(out, OUT_PATH, BASE_SHEET)
    print(f"[월 1건] 사용시트={sheets}  입력={len(work)}  출력={len(out)} → {OUT_PATH}")
    print(f"해석된 컬럼명: KW='{KW_COL}', CO='{CO_COL}', DT='{DT_COL}', URL='{URL_COL}'")

if __name__ == "__main__":
    main()


KeyError: 'G'